<a href="https://colab.research.google.com/github/Himanshu-jn52/-poem-or-novel-Classifier-AI-ML/blob/main/poemVSnovel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.DataFrame()
df1 = pd.read_csv("Poetry_vs_Novel/all.csv",usecols=["author", "content", "poem name", "age","type"])
df2 = pd.read_csv("Poetry_vs_Novel/booksummaries.txt",delimiter = "\t",header=None)
df2 = df2.drop([0,1], axis = 1)
df2.columns = ["novel name", "author", "date", "type", "content"]

for x in df1["content"][:500]:
    try:
      df = df.append([[x, 0]],ignore_index=True)
    except:
      continue

for x in df2["content"][0:500]:
    try:
      df = df.append([[x, 1]],ignore_index=True)
    except:
      continue

df.columns = ['Review', 'Sentiment']
print(df)



In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, 1000):
  review = re.sub('[^a-zA-Z]', ' ', df['Review'][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)
  print(corpus)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(corpus).toarray()
y = df.iloc[:, -1].values
print(X)
print(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)
print(X_train[0])
print(y_train[0])

In [ ]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB
#classifier = GaussianNB()
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

In [ ]:
new_review = 'Above your head no star will flame. One weary sound will be the same—the hoarse roar of the gale.The shadows fall from your tired eyes as your lone bedside candle dies, for here the calendar breeds nights till stores of candles fail.'
new_review = re.sub('[^a-zA-Z]', ' ', new_review)
new_review = new_review.lower()
new_review = new_review.split()
ps = PorterStemmer()
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')
new_review = [ps.stem(word) for word in new_review if not word in set(all_stopwords)]
new_review = ' '.join(new_review)
new_corpus = [new_review]
new_X_test = cv.transform(new_corpus).toarray()
new_y_pred = classifier.predict(new_X_test)
print(new_y_pred)

In [ ]:
new_review = '''Alex, a teenager living in near-future England, leads his gang on nightly orgies of opportunistic, random "ultra-violence." Alex's friends ("droogs" in the novel's Anglo-Russian slang, Nadsat) are: Dim, a slow-witted bruiser who is the gang's muscle; Georgie, an ambitious second-in-command; and Pete, who mostly plays along as the droogs indulge their taste for ultra-violence. Characterized as a sociopath and a hardened juvenile delinquent, Alex is also intelligent and quick-witted, with sophisticated taste in music, being particularly fond of Beethoven, or "Lovely Ludwig Van." The novel begins with the droogs sitting in their favorite hangout (the Korova Milkbar), drinking milk-drug cocktails, called "milk-plus", to hype themselves for the night's mayhem. They assault a scholar walking home from the public library, rob a store leaving the owner and his wife bloodied and unconscious, stomp a panhandling derelict, then scuffle with a rival gang. Joyriding through the countryside in a stolen car, they break into an isolated cottage and maul the young couple living there, beating the husband and raping his wife.'''
new_review = re.sub('[^a-zA-Z]', ' ', new_review)
new_review = new_review.lower()
new_review = new_review.split()
ps = PorterStemmer()
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')
new_review = [ps.stem(word) for word in new_review if not word in set(all_stopwords)]
new_review = ' '.join(new_review)
new_corpus = [new_review]
new_X_test = cv.transform(new_corpus).toarray()
new_y_pred = classifier.predict(new_X_test)
print(new_y_pred)

In [113]:
new_review = "Detonative top dressing allelic pip adequate mako longish daubentonia madagascariensis europocentric hyperlipidemia uniovulate acer platanoides undocumented semnopithecus entellus unread garonne improved family roccellaceae livid italo calvino inquisitive gunter fabian lower wing pan troglodytes schweinfurthii entitled taklamakan desert familial genus drypis platyrhinian hygiene smoggy bastinado speckless hug drug knobby patient algorithmic repeating decimal exhortative sierra leonean lxxviii recombinant protein transitory sense impression stereotypical implementation flea olympian games bowfront fishplate bibliothecarial boxer deskbound salad bar flavourless meatman lucullan auditory meatus unforethoughtful cucurbita"
new_review = re.sub('[^a-zA-Z]', ' ', new_review)
new_review = new_review.lower()
new_review = new_review.split()
ps = PorterStemmer()
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')
new_review = [ps.stem(word) for word in new_review if not word in set(all_stopwords)]
new_review = ' '.join(new_review)
new_corpus = [new_review]
new_X_test = cv.transform(new_corpus).toarray()
new_y_pred = classifier.predict(new_X_test)
print(new_y_pred)



[1]
